<a href="https://colab.research.google.com/github/dothangTH/no2_prediction/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
from osgeo import gdal  
import math 
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def truncate(f, n):
        return math.floor(f * 10 ** n) / 10 ** n 
    
def percentage(part, whole):
      return part/whole

#GEN FEATURE != SEN5P & LANDCOVER

###Define DIR, DATE, features

In [ ]:
DIR = '/content/drive/MyDrive/data_cvd/'
DATE = '20190301'
features = ['HPBL', 'PRESSURE', 'RH', 'TEMP', 'WINDSPEED', 'dew_point_01', 'dew_point_02', 'dew_point_03', 'dew_point_04', 'population_dens', 'road_dens']

<p>Đặt tên ảnh đúng định dạng: feature + '_' + DATE</p>
<p>DATE: %Y%m%d  (eg. 20190101)</p>

###Gen

In [ ]:

for feature in features:
  df = None
  df = pd.DataFrame()
  PATH = DIR + feature + '_' + DATE + '.tif'
  ds = gdal.Open(PATH, gdal.GA_ReadOnly)

  band = ds.GetRasterBand(1)
      
  cols = ds.RasterXSize
  rows = ds.RasterYSize

  transform = ds.GetGeoTransform()  

  xOrigin = transform[0]
  yOrigin = transform[3]
  pixelWidth = transform[1]
  pixelHeight = transform[5] 

  data = band.ReadAsArray(0, 0, cols, rows) 
      
  for i in range(cols):
      for j in range(rows):
          lon = x = round(xOrigin + i * pixelWidth, 3)
          lat = y = round(yOrigin + j * pixelHeight, 3)
          df = df.append({'lon': lon, 'lat': lat, feature: truncate(data[j][i], 10)}, ignore_index=True)
      print('{}: {} / {}'.format(feature, i ,cols))

  df.to_csv(feature + '_' + DATE + '.csv')

#GEN LANDCOVER

In [ ]:
# #Landcover data extracting

# PATH = '/content/drive/MyDrive/data_cvdhd/LandCover2019.tif'



# # df = pd.read_csv('HPBL.csv')
# # data_path = 'temperature/1/TMPCombine_20190101.tif' #đường dẫn tới file data
# ds = gdal.Open(PATH, gdal.GA_ReadOnly)

# band = ds.GetRasterBand(1)
    
# cols = ds.RasterXSize
# rows = ds.RasterYSize

# transform = ds.GetGeoTransform()  

# # xOrigin = transform[0]
# # yOrigin = transform[3]
# # pixelWidth = transform[1]
# # pixelHeight = transform[5] 

# xOrigin = 100.085348222534
# yOrigin = 25.7598225680898
# pixelWidth = 0.02694945852
# pixelHeight = -0.02590779389

# data = band.ReadAsArray(0, 0, cols, rows) 

# for index, row in df.iterrows():
#   print('{} / {}'.format(index, len(df)))
#   x = int((row['lon'] - xOrigin) / pixelWidth)
#   y = int((row['lat'] - yOrigin) / pixelHeight)
#   if x < cols and y < rows:
#     water = 0
#     agri = 0
#     urban = 0
#     forest = 0
#     count = 0
  

#     for i in range(max(-20, -x), min(20, cols - x - 1)):
#       for j in range(max(-20, -y), min(20, rows - y - 1)):
#           cla = data[y + j][x + i]

#           if cla == 80 or cla == 90:
#             water += 1
#           elif cla == 40:
#             agri += 1
#           elif cla == 50:
#             urban += 1
#           elif cla in range(111, 126):
#             forest += 1

#           count += 1
    
#     row['water'] = round(water / count, 10)
#     row['agri'] = round(agri / count, 10)
#     row['urban'] = round(urban / count, 10)
#     row['forest'] = round(forest / count, 10)  
  
#   else:
#     row['water'] = -9999
#     row['agri'] = -9999
#     row['urban'] = -9999
#     row['forest'] = -9999
      

#Mean dew_point

In [ ]:
df = pd.read_csv('dew_point_01_' + DATE + '.csv')
dp1 = df['dew_point_01'].values

df = pd.read_csv('dew_point_02_' + DATE + '.csv')
dp2 = df['dew_point_02'].values

df = pd.read_csv('dew_point_03_' + DATE + '.csv')
dp3 = df['dew_point_03'].values

df = pd.read_csv('dew_point_04_' + DATE + '.csv')
dp4 = df['dew_point_04'].values

In [ ]:
def mean_dew_point(dp1, dp2, dp3, dp4):
  result = []

  for i in range(len(dp1)):
    count = 0
    sum = 0

    if dp1[i] != -9999:
      count += 1
      sum += dp1[i]
    
    if dp2[i] != -9999:
      count += 1
      sum += dp2[i]

    if dp3[i] != -9999:
      count += 1
      sum += dp3[i]

    if dp4[i] != -9999:
      count += 1
      sum += dp4[i]

    if count == 0:
      result.append(-9999)
    else:
      result.append(round(sum / count, 10))

  return result

### To dew_point.csv

In [ ]:
mdp = mean_dew_point(dp1, dp2, dp3, dp4)

dp_df = pd.DataFrame()

dp_df['lon'] = df['lon']
dp_df['lat'] = df['lat']
dp_df['dew_point'] = mdp

dp_df.to_csv('dew_point_' + DATE + '.csv')

#CONCAT FEATURES DATAFRAME

##Load lon lat to df

In [ ]:
final_df = pd.DataFrame()
feat_df = pd.read_csv('HPBL_' + DATE +'.csv')

final_df['lon'] = feat_df['lon']
final_df['lat'] = feat_df['lat']

##Concat feature columns

In [ ]:
features = ['HPBL', 'PRESSURE', 'RH', 'TEMP', 'WINDSPEED', 'dew_point', 'population_dens', 'road_dens']

for feature in features:
  feat_df = pd.read_csv(feature + '_' + DATE + '.csv')
  final_df[feature] = feat_df[feature]

##Add time feature columns

In [ ]:
time = pd.to_datetime(DATE, format='%Y%m%d', errors='ignore')

In [ ]:
final_df['year'] = time.year
final_df['month'] = time.month
final_df['day'] = time.day
final_df['quarter'] = time.quarter
final_df['weekday'] = time.weekday()

##Concat landcover feature

In [ ]:
land_df = pd.read_csv(DIR + 'landcover.csv')

In [ ]:
final_df['water'] = land_df['water']
final_df['agri'] = land_df['agri']
final_df['urban'] = land_df['urban']
final_df['forest'] = land_df['forest']

#TO FINAL CSV

In [ ]:
file_name = DATE + '.csv'
final_df.to_csv(file_name)

#Satellite NO2